# Сбор и разметка данных для машинного обучения
## Домашняя работа №2. Разметка данных.

### Defenition of done
- Возьмите открытый табличный датасет, набор текстовых данных(например, отзывы о продукте) или набор изображений(можно взять из открытых источников).
есть опция, что можете взять, то что было в первом дз
- Напишите ТЗ(в свободной форме) для разметки выбранного датасета. (Для текстового датасета можно предложить разметку семантики, для набора изображений - детекцию или сегментацию). Не забывайте про примеры в ТЗ!
- Добавьте какой-либо валидатор (принимаются подмешивание тестовых кейсов/рассчет метрики согласованности - в этом случае нужно будет предостаивть разметку от минимум 2х людей).
- Отправьте ваш собранный (но неразмеченный!) датасет знакомому с курса и предложите разметить по вашему ТЗ. Получите обратную связь. Для разметки допустимо использовать одну из онлайн-платформ или самостоятельные инструменты (предоставьте инструмент вашему разметчику).
- Проанализируйте полученные результаты разметки на наличие выбросов/дубликатов/ некорректных результатов. Примените методы для их коррекции или удаления. Оцените метрику согласованности, либо квалификацию вашего разметчика на подмешанных тестовых кейсах.
- Порефлексируйте на тему процесса разметки данных и качества полученных результатов.

In [413]:
import json
import os

import numpy as np
import pandas as pd
import sklearn.metrics

In [414]:
# imgs_names = os.listdir('annotation/test/imgs')

# df = pd.read_csv('data/rtsd-dataset/dataset_signs.csv')
# df['file_name'] = df['file_name'].apply(lambda text: text.split('/')[1])

# df_temp = df[df['file_name'].isin(imgs_names)]
# df_temp.to_csv('annotation/test/rtsd_dataset_test.csv', index=False)

In [415]:
def get_target(categories):
    np_cats = np.zeros(155, dtype=int)
    for cat in categories:
        np_cats[cat - 1] = 1
        
    return np_cats

## Данные

#### Тестовые данные

In [416]:
df_test = pd.read_csv('annotation/test/rtsd_dataset_test.csv')
df_test.drop(columns=['iscrowd'], inplace=True)

df_test.head(5)

,width,height,file_name,image_id,category_id,area,bbox,category_name
0,1920,1080,autosave13_04_2013_15_59_53_2.jpg,29488,1,806,"[1107, 530, 31, 26]",2_1
1,1920,1080,autosave16_04_2013_13_10_06_3.jpg,31868,1,1400,"[961, 504, 35, 40]",2_1
2,1280,720,autosave23_10_2012_08_49_48_1.jpg,45789,1,506,"[1043, 265, 23, 22]",2_1
3,1280,720,autosave23_10_2012_09_55_19_0.jpg,47094,1,323,"[787, 358, 19, 17]",2_1
4,1280,720,autosave24_10_2012_10_36_52_4.jpg,54092,1,2400,"[639, 96, 48, 50]",2_1


In [417]:
df_test.shape

(118, 8)

In [418]:
df_test = pd.DataFrame(df_test.groupby('file_name')['category_id'].apply(list).reset_index())
df_test['target'] = df_test['category_id'].apply(get_target)

df_test.head(5)

,file_name,category_id,target
0,autosave01_02_2012_10_35_06.jpg,"[7, 7]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,autosave01_02_2012_11_29_44.jpg,"[7, 33]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,autosave01_02_2012_12_50_12.jpg,[26],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,autosave02_10_2012_12_17_18_0.jpg,"[43, 43, 43, 43]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,autosave09_10_2012_09_45_08_0.jpg,"[7, 7, 8]","[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."


#### Тестовые данные (разметчик)

In [419]:
with open('annotation/test/annotation.json') as json_file:
    data = json.load(json_file)

In [420]:
df_cats = pd.DataFrame(data['categories'])
df_cats.drop(columns=['supercategory'], inplace=True)

df_cats.head(5)

,id,name
0,1,2_1
1,2,1_23
2,3,1_17
3,4,3_24
4,5,8_2_1


In [421]:
df_imgs = pd.DataFrame(data['images'])
df_imgs.drop(columns=['license', 'flickr_url', 'coco_url', 'date_captured'], inplace=True)

df_imgs.head(5)

,id,width,height,file_name
0,1,1280,720,autosave01_02_2012_10_35_06.jpg
1,2,1280,720,autosave01_02_2012_11_29_44.jpg
2,3,1280,720,autosave01_02_2012_12_50_12.jpg
3,4,1280,720,autosave02_10_2012_12_17_18_0.jpg
4,5,1280,720,autosave09_10_2012_09_45_08_0.jpg


In [422]:
df_bboxs = pd.DataFrame(data['annotations'])
df_bboxs.drop(columns=['segmentation', 'iscrowd', 'attributes'], inplace=True)

df_bboxs.head(5)

,id,image_id,category_id,area,bbox
0,1,1,7,290.640,"[371.1, 406.2, 16.8, 17.3]"
1,2,1,7,360.800,"[631.34, 383.69, 17.6, 20.5]"
2,3,2,7,1007.920,"[730.6, 331.1, 34.4, 29.3]"
3,4,2,4,470.400,"[824.2, 382.4, 21.0, 22.4]"
4,5,2,33,5049.504,"[948.25, 141.41, 59.1, 85.44]"


In [423]:
df_temp = df_imgs.merge(df_bboxs, left_on='id', right_on='image_id')
df_temp.drop(columns=['id_x', 'id_y', 'image_id'], inplace=True)

df_annot = df_temp.merge(df_cats, left_on='category_id', right_on='id')
df_annot.drop(columns=['id'], inplace=True)

df_annot.head(5)

,width,height,file_name,category_id,area,bbox,name
0,1280,720,autosave01_02_2012_10_35_06.jpg,7,290.6400,"[371.1, 406.2, 16.8, 17.3]",5_19_1
1,1280,720,autosave01_02_2012_10_35_06.jpg,7,360.8000,"[631.34, 383.69, 17.6, 20.5]",5_19_1
2,1280,720,autosave01_02_2012_11_29_44.jpg,7,1007.9200,"[730.6, 331.1, 34.4, 29.3]",5_19_1
3,1280,720,autosave09_10_2012_09_45_08_0.jpg,7,1040.1892,"[989.36, 323.36, 32.98, 31.54]",5_19_1
4,1280,720,autosave09_10_2012_09_45_08_0.jpg,7,791.2400,"[90.6, 316.0, 30.2, 26.2]",5_19_1


In [424]:
df_annot.shape

(156, 7)

In [425]:
df_annot = pd.DataFrame(df_annot.groupby('file_name')['category_id'].apply(list).reset_index())
df_annot['target'] = df_annot['category_id'].apply(get_target)

df_annot.head(5)

,file_name,category_id,target
0,autosave01_02_2012_10_35_06.jpg,"[7, 7]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,autosave01_02_2012_11_29_44.jpg,"[7, 4, 33]","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,autosave01_02_2012_12_50_12.jpg,[26],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,autosave02_10_2012_12_17_18_0.jpg,"[43, 43, 43, 43, 22, 22, 69, 69]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,autosave09_10_2012_09_45_08_0.jpg,"[7, 7, 8, 25]","[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."


## Оценка метрик

### Подход-1

In [426]:
y_true = np.array(df_test['target'].tolist())
y_pred = np.array(df_annot['target'].tolist())

In [427]:
print('Exact Match Ratio: {0}'.format(sklearn.metrics.accuracy_score(y_true, y_pred, normalize=True, sample_weight=None)))
print('Hamming loss: {0}'.format(sklearn.metrics.hamming_loss(y_true, y_pred))) 
print('Recall: {0}'.format(sklearn.metrics.precision_score(y_true=y_true, y_pred=y_pred, average='samples'))) 
print('Precision: {0}'.format(sklearn.metrics.recall_score(y_true=y_true, y_pred=y_pred, average='samples')))
print('F1 Measure: {0}'.format(sklearn.metrics.f1_score(y_true=y_true, y_pred=y_pred, average='samples'))) 

Exact Match Ratio: 0.46
Hamming loss: 0.005806451612903226
Recall: 0.7526190476190476
Precision: 0.8966666666666666
F1 Measure: 0.8000476190476191


### Подход-2

In [428]:
df_test.head(5)

,file_name,category_id,target
0,autosave01_02_2012_10_35_06.jpg,"[7, 7]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,autosave01_02_2012_11_29_44.jpg,"[7, 33]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,autosave01_02_2012_12_50_12.jpg,[26],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,autosave02_10_2012_12_17_18_0.jpg,"[43, 43, 43, 43]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,autosave09_10_2012_09_45_08_0.jpg,"[7, 7, 8]","[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [431]:
df_annot.head(5)

,file_name,category_id,target
0,autosave01_02_2012_10_35_06.jpg,"[7, 7]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,autosave01_02_2012_11_29_44.jpg,"[7, 4, 33]","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,autosave01_02_2012_12_50_12.jpg,[26],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,autosave02_10_2012_12_17_18_0.jpg,"[43, 43, 43, 43, 22, 22, 69, 69]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,autosave09_10_2012_09_45_08_0.jpg,"[7, 7, 8, 25]","[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [430]:
# проверка наличия в annot_cats меток из test_cats

test_cats = [item.copy() for item in df_test['category_id'].tolist()]
annot_cats = [item.copy() for item in df_annot['category_id'].tolist()]

counter_nf = 0
counter_c = 0

for i in range(len(test_cats)):
    for elem in test_cats[i]:
        try:
            annot_cats[i].remove(elem)
            counter_c += 1
        except:
            print('Line:', i, 'Elem:', elem)
            counter_nf += 1

print('Count of not found:', counter_nf)
print('Count of common:', counter_c)

Line: 6 Elem: 62
Line: 10 Elem: 92
Line: 11 Elem: 3
Line: 14 Elem: 43
Line: 17 Elem: 3
Line: 19 Elem: 116
Line: 21 Elem: 110
Line: 24 Elem: 62
Line: 29 Elem: 20
Line: 30 Elem: 5
Line: 40 Elem: 52
Count of not found: 11
Count of common: 107


In [432]:
# проверка наличия в test_cats меток из annot_cats

test_cats = [item.copy() for item in df_test['category_id'].tolist()]
annot_cats = [item.copy() for item in df_annot['category_id'].tolist()]

counter_nf = 0
counter_c = 0

for i in range(len(annot_cats)):
    for elem in annot_cats[i]:
        try:
            test_cats[i].remove(elem)
            counter_c += 1
        except:
            print('Line:', i, 'Elem:', elem)
            counter_nf += 1

print('Count of not found:', counter_nf)
print('Count of common:', counter_c)

Line: 1 Elem: 4
Line: 3 Elem: 22
Line: 3 Elem: 22
Line: 3 Elem: 69
Line: 3 Elem: 69
Line: 4 Elem: 25
Line: 6 Elem: 39
Line: 6 Elem: 23
Line: 7 Elem: 33
Line: 8 Elem: 26
Line: 9 Elem: 4
Line: 9 Elem: 17
Line: 11 Elem: 7
Line: 11 Elem: 7
Line: 11 Elem: 52
Line: 14 Elem: 7
Line: 14 Elem: 7
Line: 14 Elem: 55
Line: 15 Elem: 43
Line: 15 Elem: 43
Line: 15 Elem: 43
Line: 17 Elem: 7
Line: 17 Elem: 4
Line: 17 Elem: 52
Line: 20 Elem: 13
Line: 20 Elem: 15
Line: 21 Elem: 23
Line: 24 Elem: 39
Line: 25 Elem: 7
Line: 27 Elem: 7
Line: 27 Elem: 2
Line: 27 Elem: 6
Line: 28 Elem: 129
Line: 29 Elem: 18
Line: 33 Elem: 46
Line: 35 Elem: 7
Line: 38 Elem: 26
Line: 39 Elem: 17
Line: 39 Elem: 92
Line: 40 Elem: 2
Line: 41 Elem: 7
Line: 41 Elem: 7
Line: 41 Elem: 41
Line: 45 Elem: 7
Line: 45 Elem: 69
Line: 45 Elem: 8
Line: 46 Elem: 69
Line: 47 Elem: 41
Line: 47 Elem: 41
Count of not found: 49
Count of common: 107
